In [60]:
import os
import pandas as pd
import urllib
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import numpy as np

In [32]:
path = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\data_exploration\pca\rh'

In [33]:
os.chdir(path)

fmcsv = (path + r'\fm_csv.csv')
dycsv = (path + r'\dy_csv.csv')

f = open(fmcsv,'w+')
d = open(fmcsv,'w+')

In [34]:
urllib.request.urlretrieve('http://coagmet.colostate.edu/cgi-bin/web_services.pl?type=five_minute' + 
                          '&sids=rfd01&sdate=1992-06-04&edate=2018-12-31' +
                          '&elems=tmean,rh,vp,sr,ws,wind_vec,pp,st5,st15,gust,gustdir',
                          filename=fmcsv)
fmdf = pd.read_csv(fmcsv)

In [35]:
fmdf = fmdf.reset_index()
fmdf.columns = ['station','datetime','tmean','rh','vp','sr','ws','wind_vec','pp','st5','st15',
               'gust','gustdir']

In [36]:
fmdf['datetime'] = pd.to_datetime(fmdf.datetime,errors='coerce')

idx_max = fmdf.groupby(fmdf.datetime.dt.date,sort=False)['rh'].idxmax()
idx_min = fmdf.groupby(fmdf.datetime.dt.date,sort=False)['rh'].idxmin()

fmdf_max = fmdf.loc[idx_max,]
fmdf_min = fmdf.loc[idx_min,]

In [37]:
stdev = fmdf.groupby(fmdf.datetime.dt.date,sort=False).std()

stdev = stdev.reset_index()

stdev['datetime'] = pd.to_datetime(stdev.datetime, errors='coerce')

stdev.columns = ['datetime','tmean_stdev','rh_stdev','vp_stdev','sr_stdev',
                 'ws_stdev','wind_vec_stdev','pp','st5_stdev',
                 'st15_stdev','gust_stdev','gustdir_stdev']

stdev = stdev.drop('pp',axis=1)

In [38]:
fmdf_max['datetime'] = pd.to_datetime(fmdf_max.datetime, errors='coerce')
fmdf_min['datetime'] = pd.to_datetime(fmdf_min.datetime, errors='coerce')

fmdf_max['date'] = fmdf_max.datetime.dt.date
fmdf_min['date'] = fmdf_min.datetime.dt.date

In [39]:
fmdf_max.columns = ['station','datetime_max','tmean_max','rh_max','vp_max','sr_max',
                   'ws_max','wind_vec_max','pp','st5_max','st15_max','gust_max',
                   'gustdir_max','date']

fmdf_max = fmdf_max.drop('pp',axis=1)

In [40]:
fmdf_min.columns = ['station','datetime_min','tmean_min','rh_min','vp_min','sr_min',
                   'ws_min','wind_vec_min','pp','st5_min','st15_min','gust_min',
                   'gustdir_min','date']

fmdf_min = fmdf_min.drop('pp',axis=1)

In [41]:
fmdf_mxmn = fmdf_max.merge(fmdf_min, on=fmdf_max.date)

fmdf_mxmn.columns.values[0] = 'date'
fmdf_mxmn.columns.values[1] = 'station'

stuff = ['date_x','station_y','date_y']

fmdf_mxmn = fmdf_mxmn.drop(stuff,axis=1)

In [42]:
first_obs = fmdf_mxmn.date[0]
str(first_obs)

'2016-12-13'

In [43]:
urllib.request.urlretrieve(
    'http://coagmet.colostate.edu/cgi-bin/web_services.pl?type=daily' +
    '&sids=rfd01&sdate=' + str(first_obs) + '&edate=2018-12-31&elems=' +
    'rhmax,rhmxtm,rhmin,rhmntm',filename=dycsv)

dydf = pd.read_csv(dycsv)
dydf = dydf.reset_index()
dydf.columns = ['station','date','rhmax','rhmax_time','rhmin','rhmin_time']

In [44]:
dydf = dydf.drop(dydf.tail(1).index)

In [45]:
dydf['rhmxtm'] = dydf['date'] + ' ' + dydf['rhmax_time']
dydf['rhmxtm'] = pd.to_datetime(dydf['rhmxtm'],format='%Y/%m/%d %H:%M:%S')

dydf['rhmntm'] = dydf['date'] + ' ' + dydf['rhmin_time']
dydf['rhmntm'] = pd.to_datetime(dydf['rhmntm'],format='%Y/%m/%d %H:%M:%S')

dydf['date'] = pd.to_datetime(dydf.date, errors='coerce')

In [46]:
df = fmdf_mxmn.merge(dydf,on=fmdf_mxmn.date)

In [47]:
df.columns.values[0] = 'date'
df.columns.values[2] = 'station'

stuff = ['station_y','date_y','date_x']
df = df.drop(stuff, axis=1)

In [48]:
df['rhmax_diff'] = abs(df.rh_max - df.rhmax)
df['rhmin_diff'] = abs(df.rh_min - df.rhmin)

df['rhmxtm_diff'] = abs(df.datetime_max.values - df.rhmxtm.values)
df['rhmxtm_diff'] = df['rhmxtm_diff'] / np.timedelta64(1,'m')

df['rhmntm_diff'] = abs(df.datetime_min.values - df.rhmntm.values)
df['rhmntm_diff'] = df['rhmntm_diff'] / np.timedelta64(1,'m')

In [49]:
summary = df.describe()
summary

,tmean_max,rh_max,vp_max,sr_max,ws_max,wind_vec_max,st5_max,st15_max,gust_max,gustdir_max,...,st5_min,st15_min,gust_min,gustdir_min,rhmax,rhmin,rhmax_diff,rhmin_diff,rhmxtm_diff,rhmntm_diff
count,749.000000,749.000000,749.000000,749.000000,749.000000,749.000000,749.000000,749.000000,749.000000,748.000000,...,749.000000,749.000000,749.000000,749.000000,749.000000,749.000000,749.000000,749.000000,749.000000,749.000000
mean,3.934009,0.881567,0.857244,0.760677,1.045202,176.857713,11.488377,13.426597,1.652844,206.766979,...,17.055653,14.313287,4.598131,149.651335,0.883192,0.258182,0.013299,0.015138,46.052960,25.677125
std,9.835219,0.131782,0.542611,2.182054,1.272226,120.326706,9.235922,9.120970,1.658638,98.269567,...,11.324238,9.645004,2.764295,93.257259,0.137883,0.177416,0.048826,0.014820,199.299904,82.240711
min,-22.800000,0.364000,0.087000,0.000000,0.000000,0.000000,-5.065000,-1.805000,0.000000,0.000000,...,-1.493000,-1.330000,0.000000,0.024000,0.130000,0.033000,0.000000,0.001000,0.000000,0.000000
25%,-4.537000,0.843000,0.382000,0.000000,0.122000,54.520000,2.482000,4.679000,0.800000,141.400000,...,6.852000,5.099000,2.525000,80.200000,0.848000,0.127000,0.002000,0.006000,1.500000,2.166667
50%,3.448000,0.925000,0.688000,0.000000,0.683000,199.000000,10.140000,12.240000,1.250000,230.400000,...,15.940000,13.140000,4.025000,126.900000,0.931000,0.207000,0.005000,0.012000,3.000000,4.500000
75%,13.360000,0.972000,1.311000,0.363000,1.358000,282.600000,20.940000,22.950000,2.000000,285.525000,...,28.020000,24.300000,6.050000,226.600000,0.975000,0.332000,0.011000,0.021000,5.833333,16.000000
max,23.970000,1.039000,2.261000,24.650000,8.630000,359.300000,30.560000,30.810000,10.620000,353.900000,...,38.850000,31.530000,18.580000,354.500000,1.037000,0.992000,0.612000,0.287000,1435.000000,1383.500000


In [50]:
df = df.merge(stdev,on=stdev.datetime)

In [51]:
stuff = ['date','datetime']
df = df.drop(stuff,axis=1)

In [52]:
df.columns.values[0] = 'date'

In [53]:
stuff = ['station']
df = df.drop(stuff, axis=1)

In [54]:
list(df.columns)

['date',
 'datetime_max',
 'tmean_max',
 'rh_max',
 'vp_max',
 'sr_max',
 'ws_max',
 'wind_vec_max',
 'st5_max',
 'st15_max',
 'gust_max',
 'gustdir_max',
 'datetime_min',
 'tmean_min',
 'rh_min',
 'vp_min',
 'sr_min',
 'ws_min',
 'wind_vec_min',
 'st5_min',
 'st15_min',
 'gust_min',
 'gustdir_min',
 'rhmax',
 'rhmax_time',
 'rhmin',
 'rhmin_time',
 'rhmxtm',
 'rhmntm',
 'rhmax_diff',
 'rhmin_diff',
 'rhmxtm_diff',
 'rhmntm_diff',
 'tmean_stdev',
 'rh_stdev',
 'vp_stdev',
 'sr_stdev',
 'ws_stdev',
 'wind_vec_stdev',
 'st5_stdev',
 'st15_stdev',
 'gust_stdev',
 'gustdir_stdev']

In [56]:
features = ['tmean_max','rh_max','vp_max','sr_max','ws_max','wind_vec_max',
            'st5_max','st15_max','gust_max','gustdir_max','tmean_min','rh_min',
            'vp_min','sr_min','ws_min','wind_vec_min','st5_min','st15_min','gust_min','gustdir_min',
            'rhmax','rhmin','rhmax_diff','rhmin_diff',
            'rhmxtm_diff','rhmntm_diff','tmean_stdev','rh_stdev','vp_stdev','sr_stdev','ws_stdev',
            'wind_vec_stdev','st5_stdev','st15_stdev','gust_stdev','gustdir_stdev']
df_to_standardize = df.dropna()
x = df.loc[:, features].values
y = df.loc[:,['rhmax_diff']].values

x = StandardScaler().fit_transform(x)

In [59]:
print(x)

[[-1.2986532  -0.02708871 -1.07191513 ... -1.86565615 -1.48640448
  -1.44387914]
 [-1.47059947  0.43610593 -1.10511025 ... -1.7177399  -0.3523904
  -0.18917288]
 [-1.60693558 -0.28526277 -1.19547473 ... -1.62967851 -1.36657047
   0.16269961]
 ...
 [-1.40853602  0.5424129  -1.0737593  ... -1.70935906 -1.16921732
  -0.33170244]
 [-1.65780726  0.04884484 -1.19178638 ... -1.82667375 -1.23303702
   0.4785021 ]
 [-0.42091323 -3.02646387 -1.04425254 ...         nan         nan
          nan]]


In [ ]:
pca = PCA(n_components=2)

